# Main

In [ ]:
# import the dataset from gsheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('dataset').sheet1

In [ ]:
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

import pandas as pd
df = pd.DataFrame.from_records(rows)

# creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]

# Duplicate Control

In [ ]:
import re
import pandas as pd

In [ ]:
def compare_columns(row, col1, col2):
    def clean_string(s):
        # remove all special characters and make it lowercase
        return re.sub(r'[^\w\s]', '', s).lower()

    if clean_string(row[col1]) == clean_string(row[col2]):
        return True
    else:
        return False

df['equal'] = df.apply(compare_columns, args=('reference', 't5_2'), axis=1)

In [ ]:
# another version 

def compare_columns(row, col1, col2):
    def clean_string(s):
        # remove all special characters and make it lowercase
        return re.sub(r'[^\w\s]', '', s).lower()

    if clean_string(row[col1]) == clean_string(row[col2]):
        print(f"col1 = {col1} - {row[col1]}")
        print(f"col2 = {col2} - {row[col2]}")
        return 1 
    else:
        return row[col2]

df['t5_2'] = df.apply(compare_columns, args=('reference', 't5_2'), axis=1)

col1 = reference - The hiker trained and prepared for her hikes, increasing her endurance and achieving her personal goals, feeling a sense of accomplishment and pride.
col2 = t5_2 - The hiker trained and prepared for her hikes increasing her endurance and achieving her personal goals, feeling a sense of accomplishment and pride.
col1 = reference - The Korean series Mystic Pop-up Bar is a unique and highly entertaining fantasy drama about love, fate, and the supernatural.
col2 = t5_2 - The Korean series Mystic Pop-Up Bar is a unique and highly entertaining fantasy drama about love fate and the supernatural.
col1 = reference - When planning a vacation, budget for unexpected expenses and prepare for different scenarios.
col2 = t5_2 - When planning a vacation budget for unexpected expenses and prepare for different scenarios..
col1 = reference - When saving money, set a budget, automate savings, look for ways to increase income, and invest for the future.
col2 = t5_2 - When saving money s

In [ ]:
# update the column
cell_list = worksheet.range("G2:G10002")

for i, val in enumerate(df["t5_2"]):
  cell_list[i].value = val

worksheet.update_cells(cell_list)

In [ ]:
df[df['t5_0'] == 1][['reference', 't5_0', 't5_1', 't5_2']]

,reference,t5_0,t5_1,t5_2
1066,The sound of the hammer pounding and the feeli...,1,,
3265,The Korean series Vivy: Fluorite Eye's Song 4 ...,1,,
6106,Indonesia is a destination of unique blend of ...,1,,
8944,"When buying a house, inspect, negotiate for th...",1,,


In [ ]:
# convert '1' to int/bool
def one_value(row, col):
    if row[col] == '1':
      return 1
    else: 
      return row[col]

df['t5_0'] = df.apply(lambda row: one_value(row, 't5_0'), axis=1)

In [ ]:
# df[(df['equal'] == True) & (df['equal_2'] == True)][['reference', 't5_0', 'equal', 't5_1', 'equal_1', 't5_2', 'equal_2']]

In [ ]:
# delete the comma etc which in the first place
df['chatgpt'] = df['chatgpt'].str.lstrip()

# More than 1 sentence 

In [ ]:
# more than 1 sentence

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Define a function to count the number of sentences in a cell
def count_sentences(cell):
    return len(sent_tokenize(cell))

# Apply the function to each cell in the 'text' column
df['sentence_count'] = df['chatgpt'].apply(count_sentences)

# Create a boolean column indicating whether each cell has more than one sentence
df['more_than_one_sentence'] = df['sentence_count'] > 1

# Print the rows that have more than one sentence
df[df['more_than_one_sentence'] == True].sort_values('sentence_count')